In [1]:
from utils.my_llm_utils import *
import pandas as pd

#from utils.crawler_utils import crawl_url
from utils.my_llm_utils import get_llm, validate_link, summarize_text_partnership, Config
from utils.tavily_search_utils import web_search

from urllib.parse import urlparse

import re
from datetime import datetime
import time
import os



In [2]:
# load the LLM
llm = get_llm()

# Check which llm has been loaded
if hasattr(llm, 'model'):
	print(llm.model)  # check loaded model
else:
	print(llm)

phi4:latest


In [ ]:
# # test queries

# # # test query
# # response = llm.invoke("What is the capital of Russia?")
# # print(response)

# testUrl = "https://www.crn.com/news/ai/2024/accenture-to-train-30-000-staff-on-nvidia-ai-tech-in-blockbuster-deal"



In [ ]:
# # ------------------------------------------------------
# # test search_tavily
# # ------------------------------------------------------
# search_tavily("NVIDIA AI")

# #------------------------------------------------------
# # test crawl_url
# # ------------------------------------------------------
# text = crawl_url(testUrl)


# # ------------------------------------------------------
# # test summarize_text
# # ------------------------------------------------------
# summary = summarize_text(text)
# print(summary)


In [ ]:
# query = "Accenture Nvidia partnership"
# search_response = search_tavily(query)
# print(search_response)

# text = search_response['results'][0]['content']

# summary = summarize_text(text)
# print(summary)


In [3]:
def clean_date(date_str):
    """Convert dates like 'Sep-24' to a standardized format 'yyyy-mm'"""
    if not isinstance(date_str, str) or not date_str:
        return None
    
    try:
        # Map month abbreviations to numbers
        month_map = {
            'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04',
            'May': '05', 'Jun': '06', 'Jul': '07', 'Aug': '08',
            'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12'
        }
        
        # Split the date string into month and year
        parts = date_str.strip().split('-')
        if len(parts) != 2:
            return None
            
        month_abbr, year = parts
        
        # Convert year to 4-digits (assuming 20xx for brevity)
        if len(year) == 2:
            year = f"20{year}"
            
        # Get month number
        month_num = month_map.get(month_abbr, None)
        if not month_num:
            return None
            
        return f"{year}-{month_num}"
    except:
        return None

In [4]:
def extract_date_from_text(text):
    """Try to extract a date from text using patterns"""
    # Look for common date patterns
    patterns = [
        r'(\w+\s+\d{1,2},\s*20\d{2})',  # e.g., "October 23, 2024"
        r'(\d{1,2}\s+\w+\s+20\d{2})',    # e.g., "23 October 2024"
        r'(20\d{2}-\d{1,2}-\d{1,2})'     # e.g., "2024-10-23"
    ]
    
    for pattern in patterns:
        matches = re.findall(pattern, text)
        if matches:
            try:
                date_str = matches[0]
                # Try to parse the date
                date_obj = None
                
                # Try different formats
                formats = [
                    '%B %d, %Y',      # "October 23, 2024"
                    '%b %d, %Y',      # "Oct 23, 2024"
                    '%d %B %Y',       # "23 October 2024"
                    '%d %b %Y',       # "23 Oct 2024"
                    '%Y-%m-%d'        # "2024-10-23"
                ]
                
                for fmt in formats:
                    try:
                        date_obj = datetime.strptime(date_str, fmt)
                        if date_obj:
                            # Return in month-year format
                            return f"{date_obj.strftime('%b')}-{date_obj.strftime('%y')}"
                    except:
                        continue
            except:
                pass
    return None

In [5]:
def find_partnership_info(partner1, partner2, partner3=None):
    """Search for partnership information using Tavily"""
    partners = [p for p in [partner1, partner2, partner3] if p and isinstance(p, str)]
    
    if len(partners) < 2:
        return None, None, None
    
    # Construct search query
    query = f"{partner1} {partner2} partnership announcement"
    if partner3 and isinstance(partner3, str) and partner3.strip():
        query += f" {partner3}"
        
    print(f"Searching for: {query}")
    
    # Search for the partnership info
    try:
        search_results = web_search(query, max_results=3)
        
        if not search_results or not search_results.get('results'):
            return None, None, None
            
        # Filter results to find relevant information
        relevant_results = []
        for result in search_results['results']:
            # Check if both company names are in the title or content
            title = result.get('title', '').lower()
            content = result.get('content', '').lower()
            full_text = title + ' ' + content
            
            # Check if both main partners are mentioned
            if partner1.lower() in full_text and partner2.lower() in full_text:
                # Check for words indicating partnership
                partnership_keywords = ['partner', 'collaboration', 'agreement', 'alliance', 'join']
                if any(keyword in full_text for keyword in partnership_keywords):
                    relevant_results.append(result)
        
        # Use the most relevant result or fall back to the first one
        result = relevant_results[0] if relevant_results else search_results['results'][0]
        
        link = result.get('url')
        content = result.get('content', '')
        
        # Try to extract date from content
        date = extract_date_from_text(content)
        
        # Generate summary if we have content
        summary = None
        if content and len(content.strip()) > 10:
            summary = summarize_text_partnership(content, partner1, partner2)
            
        return link, date, summary
        
    except Exception as e:
        print(f"Error searching for partnership info: {e}")
        return None, None, None

In [6]:
def update_partnerships(csv_path, output_path=None):
    """Update the AI partnerships CSV with missing information"""
    if not output_path:
        # Create output filename based on input filename
        dirname, filename = os.path.split(csv_path)
        base, ext = os.path.splitext(filename)
        output_path = os.path.join(dirname, f"{base}_updated{ext}")
    
    # Read the CSV file
    try:
        df = pd.read_csv(csv_path)
    except Exception as e:
        print(f"Error reading CSV: {e}")
        return False
    
    # Add summary column if it doesn't exist
    if 'summary' not in df.columns:
        df['summary'] = None
    
    # Process each row
    for idx, row in df.iterrows():
        partner1 = row.get('partner1')
        partner2 = row.get('partner2')
        partner3 = row.get('partner3')
        
        # Skip if we don't have at least two partners
        if not isinstance(partner1, str) or not isinstance(partner2, str):
            continue
            
        print(f"\nProcessing partnership: {partner1} and {partner2}")
        
        # Check if we have a valid link
        link1 = row.get('Link')
        link2 = row.get('link 2')
        
        valid_link = None
        content = None
        
        # Validate existing links
        if isinstance(link1, str) and validate_link(link1):
            valid_link = link1
        elif isinstance(link2, str) and validate_link(link2):
            valid_link = link2
        
        # If no valid link or missing date, search for information
        if not valid_link or pd.isna(row.get('When announced')):
            new_link, new_date, summary = find_partnership_info(partner1, partner2, partner3)
            
            # Update link if we found one
            if new_link and validate_link(new_link):
                df.at[idx, 'Link'] = new_link
                valid_link = new_link
                print(f"Found new link: {new_link}")
                
            # Update date if we found one
            if new_date and (pd.isna(row.get('When announced')) or not row.get('When announced')):
                df.at[idx, 'When announced'] = new_date
                print(f"Found new date: {new_date}")
                
            # Update summary if we found one
            if summary:
                df.at[idx, 'summary'] = summary
                print(f"Generated summary: {summary}")
            
        # If we have a valid link but no summary, generate one
        elif valid_link and (pd.isna(row.get('summary')) or not row.get('summary')):
            # Here we would fetch the content from the link, but for simplicity we'll
            # just use the link itself for the search
            print(f"Generating summary based on existing link...")
            search_results = web_search(valid_link, max_results=1)
            
            if search_results and search_results.get('results'):
                content = search_results['results'][0].get('content', '')
                if content and len(content.strip()) > 10:
                    summary = summarize_text_partnership(content, partner1, partner2)
                    df.at[idx, 'summary'] = summary
                    print(f"Generated summary: {summary}")
                    
        # Add a small delay to avoid rate limits
        time.sleep(1)
        
    # Save the updated data
    try:
        df.to_csv(output_path, index=False)
        print(f"\nUpdated data saved to {output_path}")
        return True
    except Exception as e:
        print(f"Error saving CSV: {e}")
        return False

In [9]:
# Ask for the input file path
default_path = "data/AI Partnerships sample.csv"
    
# Check if the default file exists
if not os.path.exists(default_path):
    csv_path = input(f"Enter the path to the AI partnerships CSV file (default: {default_path}): ")
    if not csv_path:
        csv_path = default_path
else:
    csv_path = default_path

In [10]:
 # Update the partnerships data
success = update_partnerships(csv_path)


Processing partnership: Hugging Face and AI Alliance
Generating summary based on existing link...
Generated summary: IBM and Meta have launched a new AI Alliance, collaborating with other tech leaders to promote the advancement of open, safe, and responsible artificial intelligence.



Processing partnership: Meta and AI Alliance
Generating summary based on existing link...
Generated summary: IBM and Meta have launched an AI Alliance, partnering with other tech leaders, to promote the development of open, safe, and responsible artificial intelligence.

**Note:** While I strive to provide accurate summaries based on given information, please verify details from official announcements for comprehensive understanding.

Processing partnership: Anthropic and Democracy works
Searching for: Anthropic Democracy works partnership announcement
Found new link: https://democracy.works/news/democracy-works-partnering-with-anthropic
Generated summary: Democracy Works is partnering with Anthropic to